        Sravya pidugu        #50249282          sravyapi@buffalo.edu

Citations:

    https://spark.apache.org/docs/2.2.0/ml-classification-regression.html
    
    
    https://mapr.com/blog/churn-prediction-pyspark-using-mllib-and-ml-packages/
    
    
    https://spark.apache.org/docs/2.1.0/ml-features.html#tf-idf
    
    

Create folders in which we are going to collect article files

In [ ]:
import urllib3,json,sys,os
from bs4 import BeautifulSoup
from topstories import TopStoriesAPI
import pandas as pd
import math
import sys, os, re
import pandas as pd
from pyspark import SparkContext, SparkConf
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from operator import add

urllib3.disable_warnings()
os.chdir('/Users/sp/Desktop/LAB3/data-analysis-using-apache-spark')
api = TopStoriesAPI('') #set API key
topic=['business','politics','sports','opinion']


def createdir(newpath):
        if not os.path.exists(newpath):
            os.makedirs(newpath)

Get article data from NYTimes using get_stories api. Save URLs into CSV and append URLs to that CSV with new URLs 
everytime you collect data,Remove duplicate URLs before navigating into the URL and parsing the data

In [ ]:

def getArticles():
    for CurrentTopic in topic: 
        createdir('./data/'+CurrentTopic)
        stories = api.get_stories(CurrentTopic)
        urls=[i["url"] for i in (stories)]
        URL_File=pd.DataFrame(urls).reset_index(drop=True)
        URL_File.to_csv(CurrentTopic+'_url_'+'.csv',mode='a',header=False) 
        URL= pd.read_csv(CurrentTopic+'_url_'+'.csv').reset_index(drop=True)
        url_list= URL.values.T.tolist()
        appended_urls=list(set(url_list[1]))
        paragraphs = []
        for url in appended_urls: #navigate to each URL and collect article data
            paragraphs.append(" ".join([i.text for i in BeautifulSoup(urllib3.connection_from_url(url).urlopen('GET',url).data, "lxml").findAll("p") if i.text != "" and "advertisement" not in i.text.lower()]).replace("\n"," "))    
            with open('./data/'+CurrentTopic+ "/article "+str(appended_urls.index(url))+".txt", "w") as text_file: #writing into a textfile
                text_file.writelines(paragraphs[appended_urls.index(url)])
                


In [223]:
getArticles()

setting variables and creating spark context

In [351]:
stop_words = set(stopwords.words('english'))
category = os.listdir("data")
category=category[1:5]
Data_folders = ["data"+"/"+i for i in category]
TOP_N = 100
sc = SparkContext.getOrCreate()

MapReduce using spark

In [352]:

lines_list = []
for i in range(len(Data_folders)):
    lines = sc.textFile(Data_folders[i])
    lines_list.append(lines)
    
    words = lines.flatMap(lambda x: x.split(" "))
    tuples= words.map(lambda x: (re.sub(r'[^a-zA-Z ]+', '',x).lower(),1))
    pairs = tuples.reduceByKey(add).collect()

Finding most frequent words

In [353]:
word_count=[]

for word,count in pairs:
    if len(word) > 2 and (word not in stop_words):
        word_count.append([word,count])
    df = pd.DataFrame(word_count,columns=["word","count"])
    Most_freq_words=[]
    df = df.sort_values('count',ascending=False)[1:]
    Most_freq_words+=(list(df[:TOP_N]["word"]))

Calculate probablity of occurrence of each word

In [354]:
def tf_idf_generator(words,feature):
    prob = float(words.count(feature.lower()))/float(len(words))
    if prob != 0.0:
        return -1.0/math.log(prob)
    else:
        return 0.0

Data tokenizing,filtering and getting features

In [355]:
def get_features(line,features,label):
    no_SpecialCharacters = re.sub('[^a-zA-Z ]','',line).lower()
    word_tokens = word_tokenize(no_SpecialCharacters)
    filtered_sentence = [w for w in word_tokens if w not in stop_words]
    TokenizedOutput = ' '.join(filtered_sentence).strip()
    words = TokenizedOutput.split()
    return [[tf_idf_generator(words,i) for i in features]+[label]]

Collect features from rdd,write it to a CSV and append labels to it

In [356]:

for i in Data_folders:
    lines = lines_list[Data_folders.index(i)]
    features = lines.flatMap(lambda x: get_features(x,Most_freq_words,i.split("/")[-1].strip().lower()))
    feature_set = features.collect()
    df = pd.DataFrame(feature_set,columns=Most_freq_words+["label"])
    label=category[Data_folders.index(i)]
    df.to_csv(label+"_features_"+".csv",index=False)
    

Append all features into a pandas DF

In [357]:
import csv
stored_data=[]
for eachSection in category:
    with open(eachSection+'_features_'+'.csv', 'r') as f:
        reader = csv.reader(f)
        lin = [i for i in reader]
        df3=pd.DataFrame(lin[1:], columns=lin[0])
        stored_data.append(df3)
df4=pd.concat(stored_data)
df4 = df4.reset_index(drop=True)  #df4 has appended features


Convert pandas dataframe into spark data frame and then to rdd

In [358]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)
sparkDF = sqlContext.createDataFrame(df4)
sparkRDD = sparkDF.rdd.map(list)

Splitting the data into 80% training data which will be used to train the model using Cross validation
The remaining 20% will be used to test the models  accuracy

In [359]:

training_data, testing_data = sparkRDD.randomSplit([0.8, 0.2])


In [360]:
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


The ML package needs data be put in a (label: Double, features: Vector) DataFrame format with correspondingly named fields. The vectorizeData() function below performs this formatting.

Next we'll pass the data through a pipeline of two transformers, StringIndexer() and VectorIndexer() which index the label and features fields respectively. Indexing categorical features allows decision trees to treat categorical features appropriately, improving performance. 

In [361]:

def vectorizeData(data):
    return data.map(lambda r: [r[-1], Vectors.dense(r[:-1])]).toDF(['label','features'])

vectorized_CV_data = vectorizeData(training_data)
vectorized_test_data = vectorizeData(testing_data)

# Index labels, adding metadata to the label column
labelIndexer = StringIndexer(inputCol='label',outputCol='indexedLabel').fit(vectorized_CV_data)

# Automatically identify categorical features and index them
featureIndexer = VectorIndexer(inputCol='features',
                               outputCol='indexedFeatures',
                               maxCategories=2).fit(vectorized_CV_data)

Training decision tree model and performing crossvalidation on it

The ML package supports k-fold cross validation, which can be readily coupled with a parameter grid builder and
an evaluator to construct a model selection workflow. 

Below, we'll use a transformation/estimation pipeline to train our models. 

The cross validator will use the ParamGridBuilder to iterate through the maxDepth parameter of the decision tree and evaluate the models using the F1-score, 
repeating 3 times per parameter value for reliable results.

In [370]:

# Train a DecisionTree model
dTree = DecisionTreeClassifier(labelCol='indexedLabel', featuresCol='indexedFeatures')

# Chain indexers and tree in a Pipeline
pipeline_DT = Pipeline(stages=[labelIndexer, featureIndexer, dTree])

# Search through decision tree's maxDepth parameter for best model
paramGrid_DT = ParamGridBuilder().addGrid(dTree.maxDepth, [2,3,4,5,6,7]).build()

# Set F-1 score as evaluation metric for best model selection
evaluator_DT = MulticlassClassificationEvaluator(labelCol='indexedLabel',
                                              predictionCol='prediction', metricName='f1')    
# Set up 3-fold cross validation
crossval_DT = CrossValidator(estimator=pipeline_DT,estimatorParamMaps=paramGrid_DT,
                             evaluator=evaluator_DT,numFolds=3)

CV_model_DT = crossval_DT.fit(vectorized_CV_data)

# Fetch best model
tree_model = CV_model_DT.bestModel.stages[2]
#print (tree_model)


The final element in our pipeline is an estimator (a decision tree classifier) 
training on the indexed labels and features.

In [379]:

transformed_data_DT = CV_model_DT.transform(vectorized_test_data)
DT_accuracy=evaluator_DT.evaluate(transformed_data)
print (evaluator.getMetricName(), DT_accuracy)

#predictions_DT = transformed_data_DT.select('indexedLabel', 'prediction', 'probability')

#predictions_DT.toPandas().head()


accuracy 0.7616747049660055


RandomForest model

In [380]:
# Train a RandomForest model
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Chain indexers and tree in a Pipeline
pipeline_RF = Pipeline(stages=[labelIndexer, featureIndexer, rf])

# Search through decision tree's maxDepth parameter for best model
paramGrid_RF = ParamGridBuilder().addGrid(rf.maxDepth, [2,3,4,5,6,7]).build()

# Set F-1 score as evaluation metric for best model selection
evaluator_RF = MulticlassClassificationEvaluator(labelCol='indexedLabel',
                                              predictionCol='prediction', metricName='f1')    

# Set up 3-fold cross validation
crossval_RF = CrossValidator(estimator=pipeline_RF,estimatorParamMaps=paramGrid_RF,
                          evaluator=evaluator_RF,
                          numFolds=3)

CV_model_RF = crossval_RF.fit(vectorized_CV_data)

# Fetch best model
rf_model = CV_model_RF.bestModel.stages[2]
#print (rf_model)


In [381]:

transformed_data_RF = CV_model_RF.transform(vectorized_test_data)
RF_accuracy=evaluator.evaluate(transformed_data_RF)
print ('accuracy:', RF_accuracy)

predictions_rf = transformed_data_RF.select('indexedLabel', 'prediction', 'probability')
#predictions_rf.toPandas().head()


accuracy: 0.7652173913043478


In [390]:
if RF_accuracy>DT_accuracy:
    print ('Random forest is a better model for the given data')
else:
    print ('Decision tree is a better model for the given data')

Random forest is a better model for the given data


Testing area

In [388]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# Train a RandomForest model.
randomForest = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, randomForest, labelConverter])

# Train model.  This also runs the indexers.
modelRF = pipeline.fit(vectorized_CV_data)

# Make predictions.
predictions = modelRF.transform(vectorized_test_data)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
##

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, randomForest])

# Search through decision tree's maxDepth parameter for best model
paramGrid = ParamGridBuilder().addGrid(randomForest.maxDepth, [2,3,4,5,6,7]).addGrid(randomForest.maxBins,[12,23,32,45]).build()
# Set up 3-fold cross validation
crossval = CrossValidator(estimator=pipeline,estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

CV_model_RF = crossval.fit(vectorized_CV_data)

# Fetch best model
RF_model_CV = CV_model_RF.bestModel.stages[2]
#print (RF_model_CV)
vectorized_test_data = vectorizeData(testing_data)
transformed_data = CV_model_RF.transform(vectorized_test_data)
print (evaluator.getMetricName(), 'accuracy:', evaluator.evaluate(transformed_data))
#rfModel = modelRF.stages[2]
#print(rfModel)  # summary only

+--------------+-------+--------------------+
|predictedLabel|  label|            features|
+--------------+-------+--------------------+
|       opinion|opinion|[0.15173813649584...|
|       opinion|opinion|[0.23844845638038...|
|       opinion|opinion|[0.20522715403771...|
|       opinion|opinion|[0.0,0.0,0.297313...|
|       opinion|opinion|[0.0,0.0,0.0,0.33...|
+--------------+-------+--------------------+
only showing top 5 rows

accuracy accuracy: 0.7652173913043478
